<a href="https://colab.research.google.com/github/lazycustard/PawDetect-CNN-for-Pet-Recognition/blob/main/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

zip_path = '/content/drive/MyDrive/dataset.zip'
extract_path = '/content/drive/MyDrive/dataset'

import zipfile
import os

if not os.path.exists(extract_path):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)
    print("✅ Files extracted to Google Drive.")
else:
    print("✅ Extraction skipped — folder already exists.")

print("📁 Extracted folders/files:")
print(os.listdir(extract_path))

train_path = '/content/drive/MyDrive/dataset/dataset/training_set'
test_path = '/content/drive/MyDrive/dataset/dataset/test_set'


print(f"\n📂 Inside training folder:")
print(os.listdir('/content/drive/MyDrive/dataset/dataset/training_set'))
print(f"\n📂 Inside test folder:")
print(os.listdir('/content/drive/MyDrive/dataset/dataset/test_set'))


from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)

training_set = train_datagen.flow_from_directory(
    train_path,
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary'
)

test_set = test_datagen.flow_from_directory(
    test_path,
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary'
)

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

classifier = Sequential()
classifier.add(Conv2D(32, (3, 3), input_shape=(64, 64, 3), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))
classifier.add(Conv2D(32, (3, 3), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))
classifier.add(Flatten())
classifier.add(Dense(units=128, activation='relu'))
classifier.add(Dense(units=1, activation='sigmoid'))

classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

classifier.fit(
    training_set,
    steps_per_epoch=len(training_set),
    epochs=10,
    validation_data=test_set,
    validation_steps=len(test_set)
)

from google.colab import files
uploaded = files.upload()  # Upload a single image

img_path = list(uploaded.keys())[0]

from tensorflow.keras.preprocessing import image
import numpy as np

test_image = image.load_img(img_path, target_size=(64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis=0)
test_image /= 255.0  # Normalize

result = classifier.predict(test_image)

print("Class Indices:", training_set.class_indices)

if result[0][0] > 0.5:
    prediction = 'dog'
else:
    prediction = 'cat'

print(f"🧠 Prediction: This image is a {prediction}.")



Mounted at /content/drive
✅ Extraction skipped — folder already exists.
📁 Extracted folders/files:
['dataset']

📂 Inside training folder:
['.DS_Store', 'cats', 'dogs']

📂 Inside test folder:
['.DS_Store', 'cats', 'dogs']
Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 170s 434ms/step - accuracy: 0.5643 - loss: 0.6760 - val_accuracy: 0.6795 - val_loss: 0.6000
Epoch 2/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 130s 381ms/step - accuracy: 0.6785 - loss: 0.5966 - val_accuracy: 0.7305 - val_loss: 0.5514
Epoch 3/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 146s 399ms/step - accuracy: 0.7022 - loss: 0.5586 - val_accuracy: 0.7580 - val_loss: 0.5153
Epoch 4/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 136s 374ms/step - accuracy: 0.7409 - loss: 0.5253 - val_accuracy: 0.7600 - val_loss: 0.5152
Epoch 5/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 142s 374ms/step - accuracy: 0.7624 - loss: 0.4894 - val_accuracy: 0.7535 - val_loss: 0.5098
Epoch 6/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 100s 400ms/step - accuracy: 0.7645 - loss: 0.4878 - val_accuracy: 0.7675 - val_loss: 0.4861
Epoch 7/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 138s 383ms/step - accuracy: 0.7731 - loss: 0.4726 - val_accuracy: 0.7780 - val_loss: 0.4698
Epoch 8/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 142s 384ms/step - accuracy: 0.7860 -

Saving catss.jpg to catss.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
Class Indices: {'cats': 0, 'dogs': 1}
🧠 Prediction: This image is a cat.
